# Gurobi multiple ml models
Optimize the costs of chemicals using gurobi ml models

## Root folder and read env variables

In [1]:
import os
# fix root path to save outputs
actual_path = os.path.abspath(os.getcwd())
list_root_path = actual_path.split('\\')[:-1]
root_path = '\\'.join(list_root_path)
os.chdir(root_path)
print('root path: ', root_path)

root path:  D:\github-mi-repo\Optimization-Industrial-Process


In [2]:
import os
from dotenv import load_dotenv, find_dotenv # package used in jupyter notebook to read the variables in file .env

""" get env variable from .env """
load_dotenv(find_dotenv())

""" Read env variables and save it as python variable """
PROJECT_GCP = os.environ.get("PROJECT_GCP", "")

## RUN

In [3]:
import pickle
import pandas as pd
import numpy as np

#gurobi
import gurobipy_pandas as gppd
from gurobi_ml import add_predictor_constr
import gurobipy as gp

## PREPARATION

### 1. Load data needs to use

In [4]:
path_data = 'artifacts/data/data.pkl'
data = pd.read_pickle(path_data)
data.head()

,230AIT446.PNT,240AIC022.MEAS,240AIC126.MEAS,240AIC224.MEAS,240AIC286.MEAS,240AIC324.MEAS,240AIC433.MEAS,240AIT063A.PNT,240AIT063B.PNT,240AIT225A.PNT,...,S240ALDP022,S240ALDP031,S240ALDP032,S276PER002,S2MAQUINAT07,S76ALE017,SSTRIPPING015,calc_prod_d0,calc_prod_d1,calc_prod_p
datetime,,,,,,,,,,,,,,,,,,,,,
2021-01-01 00:05:00,11.555040,2.983948,11.346645,4.413519,4.352375,10.441675,4.292521,5.869320,62.37495,1.837519,...,91.49,1.8,11.4,11.77,1.5712,173.6,964.0,3240.8635,3313.6215,3259.3745
2021-01-01 00:10:00,11.552320,3.015669,11.353215,4.413179,4.347186,10.432170,4.289684,5.869320,62.37495,1.814020,...,91.49,1.8,11.4,11.77,1.5712,173.6,964.0,3260.7475,3301.6920,3208.6785
2021-01-01 00:15:00,11.549955,3.018903,11.355525,4.408321,4.355828,10.410115,4.284427,5.869320,62.37495,1.814020,...,91.49,1.8,11.4,11.77,1.5712,173.6,964.0,3265.5765,3284.1330,3210.7790
2021-01-01 00:20:00,11.547145,3.001164,11.326725,4.408659,4.361292,10.379145,4.285478,5.835750,62.37495,1.814020,...,91.49,1.7,11.3,11.77,1.5712,173.6,964.0,3253.7750,3271.9260,3221.7745
2021-01-01 00:25:00,11.543160,3.017393,11.336345,4.408596,4.356374,10.387205,4.304148,5.802179,62.37495,1.814020,...,91.49,1.6,11.2,11.77,1.5712,173.6,964.0,3236.9790,3267.3050,3227.6935


### 2. Load model machine learning - MANUALLY - HARDCODED

In [5]:
######################## read model d0eop_microkappa ########################
path_model_d0eop_microkappa = f'artifacts/models/d0eop_microkappa/lr.pkl'
model_d0eop_microkappa = pd.read_pickle(path_model_d0eop_microkappa)
model_d0eop_microkappa

Pipeline(steps=[('poly_feature_2', PolynomialFeatures()),
                ('lr', LinearRegression())])

In [6]:
######################## read model d1_brillo ########################
path_model_d1_brillo = f'artifacts/models/d1_brillo/lr.pkl'
model_d1_brillo = pd.read_pickle(path_model_d1_brillo)
model_d1_brillo

LinearRegression()

In [7]:
######################## read model p_blancura ########################
path_model_p_blancura = f'artifacts/models/p_blancura/lr.pkl'
model_p_blancura = pd.read_pickle(path_model_p_blancura)
model_p_blancura

Pipeline(steps=[('poly_feature_2', PolynomialFeatures()),
                ('lr', LinearRegression())])

### 3. Define list of features and target for each model
Load master table of configuration and load data

In [8]:
######## MODEL: 'd0eop_microkappa' ########

# read table master tag
name_model = 'd0eop_microkappa'
path_list_features_target_to_optimization = f'config/optimization_engine/ml_models/MaestroTags-{name_model}-general.xlsx'
master_tags = pd.read_excel(path_list_features_target_to_optimization)

# define lists: "list_features", "list_features_controlables", "list_target"
list_features_d0eop_microkappa = master_tags[master_tags['CLASIFICACION']!= 'R']['TAG'].tolist()
list_features_controlables_d0eop_microkappa = master_tags[master_tags['CLASIFICACION'] == 'C']['TAG'].tolist()
list_target_d0eop_microkappa = master_tags[master_tags['CLASIFICACION'] == 'R']['TAG'].tolist()

# print master tags
master_tags

,TAG,TAG_DESCRIPTION,DESCRIPCION,ETAPA,CLASIFICACION,USE_PREVIOUS_MODEL,USE_NEXT_MODEL
0,calc_prod_d0,calc_prod_d0,Producción entrada D0 (prod entrada A dezplazada),D0,NC,NaN,NaN
1,240AIC022.MEAS,ph_a,pH entrada etapa Acida,A,NC,NaN,NaN
2,240AIT063B.PNT,brillo_d0,Brillo salida etapa A (entrada D0),D0,NC,NaN,NaN
3,240AIT063A.PNT,kappa_d0,Kappa salida etapa A (entrada D0),D0,NC,NaN,NaN
4,S276PER002,concentracion_clo2_d0,Concentración ClO2,D0,NC,NaN,NaN
5,SSTRIPPING015,dqo_evaporadores,DQO Evaporadores,D0,NC,NaN,NaN
6,240FY050.RO02,especifico_dioxido_d0,Específico ClO2,D0,C,NaN,NaN
7,240FY11PB.RO01,especifico_peroxido_eop,Esp. Peróxido,EOP,C,NaN,NaN
8,240FY118B.RO01,especifico_oxigeno_eop,Esp. Oxígeno,EOP,C,NaN,NaN
9,240FY107A.RO01,especifico_soda_eop,Esp. Soda EOP,EOP,C,NaN,NaN


In [9]:
######## MODEL: 'd1_brillo' ########

# read table master tag
name_model = 'd1_brillo'
path_list_features_target_to_optimization = f'config/optimization_engine/ml_models/MaestroTags-{name_model}-general.xlsx'
master_tags = pd.read_excel(path_list_features_target_to_optimization)

# define lists: "list_features", "list_features_controlables", "list_target"
list_features_d1_brillo = master_tags[master_tags['CLASIFICACION']!= 'R']['TAG'].tolist()
list_features_controlables_d1_brillo = master_tags[master_tags['CLASIFICACION'] == 'C']['TAG'].tolist()
list_target_d1_brillo = master_tags[master_tags['CLASIFICACION'] == 'R']['TAG'].tolist()

# print master tags
master_tags

,TAG,TAG_DESCRIPTION,DESCRIPCION,ETAPA,CLASIFICACION,USE_PREVIOUS_MODEL,USE_NEXT_MODEL
0,240FI108A.PNT,prod_bypass,Producción by pass,D1,NC,NaN,NaN
1,calc_prod_d1,calc_prod_d1,Producción entrada D1 (prod entrada A desplazada),D1,NC,NaN,NaN
2,240AIT225A.PNT,microkappa_d1,mKappa salida EOP (entrada D1 ),D1,NC,R - D0EOP,NaN
3,240AIT225B.PNT,blancura_d1,Blancura salida EOP (entrada D1),D1,NC,R - D0EOP,NaN
4,240TIT223.PNT,temperatura_d1,T° entrada D1,D1,NC,NaN,NaN
5,240FY218.RO02,especifico_dioxido_d1,Esp Dióxido,D1,C,NaN,NaN
6,240FY210A.RO01,especifico_acido_d1,Esp Acido,D1,C,NaN,NaN
7,240AIT322B.PNT,brillo_entrada_p,Brillo salida D1 (entrada P),P,R,NaN,NC - P


In [10]:
######## MODEL: 'p_blancura' ########

# read table master tag
name_model = 'p_blancura'
path_list_features_target_to_optimization = f'config/optimization_engine/ml_models/MaestroTags-{name_model}-general.xlsx'
master_tags = pd.read_excel(path_list_features_target_to_optimization)

# define lists: "list_features", "list_features_controlables", "list_target"
list_features_p_blancura = master_tags[master_tags['CLASIFICACION']!= 'R']['TAG'].tolist()
list_features_controlables_p_blancura = master_tags[master_tags['CLASIFICACION'] == 'C']['TAG'].tolist()
list_target_p_blancura = master_tags[master_tags['CLASIFICACION'] == 'R']['TAG'].tolist()

# print master tags
master_tags

,TAG,TAG_DESCRIPTION,DESCRIPCION,ETAPA,CLASIFICACION,USE_PREVIOUS_MODEL,USE_NEXT_MODEL
0,calc_prod_p,calc_prod_p,Producción entrada P (prod entrada A desplazada),P,NC,NaN,NaN
1,240AIT322B.PNT,brillo_p,Brillo salida D1 (entrada P),P,NC,R - D1,NaN
2,240AIC324.MEAS,ph_p,pH entrada P,P,NC,R - D1,NaN
3,240FY397.RO01,especifico_peroxido_p,Esp Peróxido P,P,C,NaN,NaN
4,240FY312.RO01,especifico_soda_p,Esp Soda,P,C,NaN,NaN
5,240FY430.RO01,especifico_acido_p,Esp Acido tac blanca,P,C,NaN,NaN
6,240AIT416B.PNT,blancura_salida_p,Blancura Salida P linea,TAC,R,NaN,NaN


### 4. Load tables parameters for optimization - bounds

In [11]:
# load bounds of decision variables in optimization that also are FEATURES in machine learning models

path_bounds_decision_var_features = 'config/optimization_engine/config_optimization/Bounds-DecisionVar-Features-x.xlsx'
bounds_decision_var_features = pd.read_excel(path_bounds_decision_var_features)
bounds_decision_var_features

,MODEL,TAG,TAG_DESCRIPTION,DESCRIPCION,MIN_VALUE,MAX_VALUE
0,d0eop_microkappa,240FY050.RO02,especifico_dioxido_d0,Específico ClO2,2,11
1,d0eop_microkappa,240FY118B.RO01,especifico_oxigeno_eop,Esp. Oxígeno,0,2
2,d0eop_microkappa,240FY11PB.RO01,especifico_peroxido_eop,Esp. Peróxido,1,15
3,d0eop_microkappa,240FY107A.RO01,especifico_soda_eop,Esp. Soda EOP,4,20
4,d1_brillo,240FY218.RO02,especifico_dioxido_d1,Esp Dióxido,1,10
5,d1_brillo,240FY210A.RO01,especifico_acido_d1,Esp Acido,0,10
6,p_blancura,240FY312.RO01,especifico_soda_p,Esp Soda,0,10
7,p_blancura,240FY397.RO01,especifico_peroxido_p,Esp Peróxido P,0,10
8,p_blancura,240FY430.RO01,especifico_acido_p,Esp Acido tac blanca,0,10


In [12]:
# load bounds of decision variables in optimization that also are TARGETS in machine learning models
# obs: the targets of one model are features in the next model. They are defined once

path_bounds_decison_var_target = 'config/optimization_engine/config_optimization/Bounds-DecisionVar-Target-y.xlsx'
bounds_decison_var_target = pd.read_excel(path_bounds_decison_var_target)
bounds_decison_var_target

,MODEL,TAG,TAG_DESCRIPTION,DESCRIPCION,MIN_VALUE,MAX_VALUE
0,d0eop_microkappa,240AIT225A.PNT,microkappa_d1,mKappa salida EOP (entrada D1 ),1.9,2.4
1,d1_brillo,240AIT322B.PNT,brillo_entrada_p,Brillo salida D1 (entrada P),89.2,100.0
2,p_blancura,240AIT416B.PNT,blancura_salida_p,Blancura Salida P linea,90.0,91.0


In [13]:
bounds_decison_var_target[bounds_decison_var_target['TAG_DESCRIPTION'] == 'microkappa_d1']['MIN_VALUE'].values[0]

1.9

### 5. Load Prices
Parameters in Optimization

In [14]:
path_prices = 'config/optimization_engine/config_optimization/price-chemicals.xlsx'
prices = pd.read_excel(path_prices)
prices

,acido,peroxido,soda,oxigeno,dioxido
0,0.275,0.698,0.655,0.092,1.4


## RUN OPTIMIZATION

### 0. Load transversal params - sets of optimization model
Transversal all codes, not only this code. For example order in features in the data.

Save the sets of optimization model as pandas index

In [15]:
# define set to models. In this example there is a set with only one element
list_bleaching = ['bleaching']
index_bleaching = pd.Index(list_bleaching)
index_bleaching

Index(['bleaching'], dtype='object')

### 1. Create guroby optimization model
Documentation: https://www.gurobi.com/documentation/current/refman/py_model.html

In [16]:
# env = gp.Env(params=params)

#Create the model within the Gurobi environment
m = gp.Model(name = "Bleaching Optimization v1")

Restricted license - for non-production use only - expires 2025-11-24


### 2. Upper bounds and lower bounds of decision variables

### 3. Input parameters of optimization model
##### That are not decision variables either parameters of machine learning model)

In [17]:
# price parameters
prices

,acido,peroxido,soda,oxigeno,dioxido
0,0.275,0.698,0.655,0.092,1.4


### 4. Features input machine learning model fixed (that are not decision variables or parameters in optimization model)
Define the features that are inputs of machine learning model that are not decision variables of optimization model (so this values doesn't change). And also, this features that are not parameters of optimization model, so this values are not used in the restrictions

**Generar una instancia a partir de los datos para las feautures de los modelos de ML que no son variables de decisión ni targets. La instancia generada se hace a partir de los datos. Para este ejemplo se va a considerar la media**

In [18]:
######################## generate instance for model d0eop_microkappa ########################

# list feature NC
list_features_d0eop_microkappa_no_vc = list(set(list_features_d0eop_microkappa) - set(list_features_controlables_d0eop_microkappa))

# generate dataframe with the mean
instance_no_controlables_d0eop_microkappa = data[list_features_d0eop_microkappa_no_vc].mean().to_frame().T
instance_no_controlables_d0eop_microkappa

,SSTRIPPING015,240AIC022.MEAS,240AIT063B.PNT,calc_prod_d0,240AIT063A.PNT,S276PER002
0,707.378644,2.938413,61.824864,3347.509416,6.349621,11.516687


In [19]:
######################## generate instance for model d1_brillo ########################

# list features NC
list_features_d1_brillo_no_vc = list(set(list_features_d1_brillo) - set(list_features_controlables_d1_brillo)) # substract vc d1_brillo
list_features_d1_brillo_no_vc = list(set(list_features_d1_brillo_no_vc) - set(list_target_d0eop_microkappa))# substract target d0eop_microkappa

# generate dataframe with the mean
instance_no_controlables_d1_brillo = data[list_features_d1_brillo_no_vc].mean().to_frame().T
instance_no_controlables_d1_brillo

,240AIT225B.PNT,240TIT223.PNT,calc_prod_d1,240FI108A.PNT
0,85.050203,77.214156,3348.667487,169.548609


In [20]:
######################## generate instance for model p_blancura ########################

# list features NC
list_features_p_blancura_no_vc = list(set(list_features_p_blancura) - set(list_features_controlables_p_blancura)) # substract vc p_blancura
list_features_p_blancura_no_vc = list(set(list_features_p_blancura_no_vc) - set(list_target_d1_brillo))# substract target d1_brillo

# generate dataframe with the mean
instance_no_controlables_p_blancura = data[list_features_p_blancura_no_vc].mean().to_frame().T
instance_no_controlables_p_blancura

,calc_prod_p,240AIC324.MEAS
0,3347.782721,10.382128


IMPORTANT: En los modelos de las etapas siguientes se deben borrar las features que son variables controlables (variables de decisión optimizador) y además los target de etapas previas que se vuelven variables no controlables de la etapa siguiente (ya que también son variables de decisión del optimizador)

### 5. Decision variables of optimization model

Let us now define the decision variables. In our model, we want to store the price and number of avocados allocated to each region. We also want variables that track how many avocados are predicted to be sold and how many are predicted to be wasted. 

All those variables are created using gurobipy-pandas, with the function `gppd.add_vars`. To use this function it is necessary to define:
- model: optimization model of gurobi
- index: pandas index. With this index it can defined the sets of the decision variables
- name: name of the decision variable
- Example: x = gppd.add_vars(model, index, name="x")

In [21]:
######################## decision variables that are FEATURES in Machiine Learning Models ########################

# model d0eop_microkappa
especifico_dioxido_d0 = gppd.add_vars(m, index_bleaching, name = "especifico_dioxido_d0", 
                                      lb = bounds_decision_var_features[bounds_decision_var_features['TAG_DESCRIPTION'] == 'especifico_dioxido_d0']['MIN_VALUE'].values[0], 
                                      ub = bounds_decision_var_features[bounds_decision_var_features['TAG_DESCRIPTION'] == 'especifico_dioxido_d0']['MAX_VALUE'].values[0]
                                     )

especifico_oxigeno_eop = gppd.add_vars(m, index_bleaching, name = "especifico_oxigeno_eop", 
                                      lb = bounds_decision_var_features[bounds_decision_var_features['TAG_DESCRIPTION'] == 'especifico_oxigeno_eop']['MIN_VALUE'].values[0], 
                                      ub = bounds_decision_var_features[bounds_decision_var_features['TAG_DESCRIPTION'] == 'especifico_oxigeno_eop']['MAX_VALUE'].values[0]
                                     )

especifico_peroxido_eop = gppd.add_vars(m, index_bleaching, name = "especifico_peroxido_eop", 
                                      lb = bounds_decision_var_features[bounds_decision_var_features['TAG_DESCRIPTION'] == 'especifico_peroxido_eop']['MIN_VALUE'].values[0], 
                                      ub = bounds_decision_var_features[bounds_decision_var_features['TAG_DESCRIPTION'] == 'especifico_peroxido_eop']['MAX_VALUE'].values[0]
                                     )

especifico_soda_eop = gppd.add_vars(m, index_bleaching, name = "especifico_soda_eop", 
                                      lb = bounds_decision_var_features[bounds_decision_var_features['TAG_DESCRIPTION'] == 'especifico_soda_eop']['MIN_VALUE'].values[0], 
                                      ub = bounds_decision_var_features[bounds_decision_var_features['TAG_DESCRIPTION'] == 'especifico_soda_eop']['MAX_VALUE'].values[0]
                                     )



# model d1_brillo
especifico_dioxido_d1 = gppd.add_vars(m, index_bleaching, name = "especifico_dioxido_d1", 
                                      lb = bounds_decision_var_features[bounds_decision_var_features['TAG_DESCRIPTION'] == 'especifico_dioxido_d1']['MIN_VALUE'].values[0], 
                                      ub = bounds_decision_var_features[bounds_decision_var_features['TAG_DESCRIPTION'] == 'especifico_dioxido_d1']['MAX_VALUE'].values[0]
                                     )

especifico_acido_d1 = gppd.add_vars(m, index_bleaching, name = "especifico_acido_d1", 
                                      lb = bounds_decision_var_features[bounds_decision_var_features['TAG_DESCRIPTION'] == 'especifico_acido_d1']['MIN_VALUE'].values[0], 
                                      ub = bounds_decision_var_features[bounds_decision_var_features['TAG_DESCRIPTION'] == 'especifico_acido_d1']['MAX_VALUE'].values[0]
                                     )



# model p_blancura
especifico_soda_p = gppd.add_vars(m, index_bleaching, name = "especifico_soda_p", 
                                      lb = bounds_decision_var_features[bounds_decision_var_features['TAG_DESCRIPTION'] == 'especifico_soda_p']['MIN_VALUE'].values[0], 
                                      ub = bounds_decision_var_features[bounds_decision_var_features['TAG_DESCRIPTION'] == 'especifico_soda_p']['MAX_VALUE'].values[0]
                                     )

especifico_peroxido_p = gppd.add_vars(m, index_bleaching, name = "especifico_peroxido_p", 
                                      lb = bounds_decision_var_features[bounds_decision_var_features['TAG_DESCRIPTION'] == 'especifico_peroxido_p']['MIN_VALUE'].values[0], 
                                      ub = bounds_decision_var_features[bounds_decision_var_features['TAG_DESCRIPTION'] == 'especifico_peroxido_p']['MAX_VALUE'].values[0]
                                     )

especifico_acido_p = gppd.add_vars(m, index_bleaching, name = "especifico_acido_p", 
                                      lb = bounds_decision_var_features[bounds_decision_var_features['TAG_DESCRIPTION'] == 'especifico_acido_p']['MIN_VALUE'].values[0], 
                                      ub = bounds_decision_var_features[bounds_decision_var_features['TAG_DESCRIPTION'] == 'especifico_acido_p']['MAX_VALUE'].values[0]
                                     )

In [22]:
######################## decision variables that are TARGETS in Machiine Learning Models ########################

# model d0eop_microkappa
microkappa_d1 = gppd.add_vars(m, index_bleaching, name = "microkappa_d1", 
                                      lb = bounds_decison_var_target[bounds_decison_var_target['TAG_DESCRIPTION'] == 'microkappa_d1']['MIN_VALUE'].values[0], 
                                      ub = bounds_decison_var_target[bounds_decison_var_target['TAG_DESCRIPTION'] == 'microkappa_d1']['MAX_VALUE'].values[0]
                                     )

# model d1_brillo
brillo_entrada_p = gppd.add_vars(m, index_bleaching, name = "brillo_entrada_p", 
                                      lb = bounds_decison_var_target[bounds_decison_var_target['TAG_DESCRIPTION'] == 'brillo_entrada_p']['MIN_VALUE'].values[0], 
                                      ub = bounds_decison_var_target[bounds_decison_var_target['TAG_DESCRIPTION'] == 'brillo_entrada_p']['MAX_VALUE'].values[0]
                                     )
# model p_blancura
blancura_salida_p = gppd.add_vars(m, index_bleaching, name = "blancura_salida_p", 
                                      lb = bounds_decison_var_target[bounds_decison_var_target['TAG_DESCRIPTION'] == 'blancura_salida_p']['MIN_VALUE'].values[0], 
                                      ub = bounds_decison_var_target[bounds_decison_var_target['TAG_DESCRIPTION'] == 'blancura_salida_p']['MAX_VALUE'].values[0]
                                     )

### 6. Constraints (constraints that are not generated by a ml model)

In [23]:
m

<gurobi.Model Continuous instance Bleaching Optimization v1: 0 constrs, 0 vars, No parameter changes>

In [24]:
# update model
m.update()

### 7. Add constraints that are machine learning models
To add constraints that have machine learning models it is necessary define a dataframe that are the instance of prediction (it has columns as gurobi decision variables) and then create the constraint in gurobi.

In this example, where each region has its own model, the dataframe instance also needs to be defined indidually. For the decision variable that are defined in the set "regions" it is important filter the dataframe instance with the correct element of the set region

**So, for each element in set region will be defined the instance dataframe and a constraint. Each region has it own model**Also, the instance has only one row, so now it is possible define a optimization model with set "time" and each row of the dataframe could be the instance of time t, t+1, t+2, etc


**IMPORTANT: LOGICALLY, FOR THIS EXAMPLE, TO DEFINE THE CONSTRAINTS OF ML MODELS, A FOR COULD HAVE BEEN MADE IN THE SET "REGIONS" BUT IT WAS NOT DONE CONSCIOUSLY THINKING OF AN EXAMPLE IN WHICH RESTRICTIONS HAVE TO BE DEFINED IN DIFFERENT SETS**

In [25]:
## show list elements in sets
index_bleaching

Index(['bleaching'], dtype='object')

#### 7.1 d0eop_microkappa

In [26]:
######################## instance model d0eop_microkappa ########################

# create instance with controlables variables. sorted according the list of features. ES MUY IMPORTANTE QUE ESTÉ ORDENADO LAS VARIABLES DE DECUISIÓN DE ACUERDO A LA LISTA DE FEATURES
instance_controlables_d0eop_microkappa = pd.DataFrame([especifico_dioxido_d0, especifico_oxigeno_eop, especifico_peroxido_eop, especifico_soda_eop]).T
instance_controlables_d0eop_microkappa.columns = list_features_controlables_d0eop_microkappa # rename columns
instance_controlables_d0eop_microkappa.reset_index(inplace = True)
instance_controlables_d0eop_microkappa.drop(columns = 'index', inplace = True)

# append features controlables with no controlables
instance_d0eop_microkappa = pd.concat([instance_no_controlables_d0eop_microkappa, instance_controlables_d0eop_microkappa], axis = 1)
instance_d0eop_microkappa = instance_d0eop_microkappa[list_features_d0eop_microkappa] # sort features

# set index - optimization set
instance_d0eop_microkappa.index = index_bleaching

instance_d0eop_microkappa

,calc_prod_d0,240AIC022.MEAS,240AIT063B.PNT,240AIT063A.PNT,S276PER002,SSTRIPPING015,240FY050.RO02,240FY11PB.RO01,240FY118B.RO01,240FY107A.RO01
bleaching,3347.509416,2.938413,61.824864,6.349621,11.516687,707.378644,<gurobi.Var especifico_dioxido_d0[bleaching]>,<gurobi.Var especifico_oxigeno_eop[bleaching]>,<gurobi.Var especifico_peroxido_eop[bleaching]>,<gurobi.Var especifico_soda_eop[bleaching]>


In [27]:
# ##### DEBUGGING - SE PODRIA AGREGAR COMO UN ASSERT PARA ASEGURAR QUE LA INSTANCIA ESTÁ BIEN CREADA
# # load intance example getting during train
# name_model = 'd0eop_microkappa'
# path_instance_d0eop_microkappa_example_train = f'config/optimization_engine/ml_models/{name_model}-example-input-model.xlsx'
# instance_d0eop_microkappa_example_train = pd.read_excel(path_instance_d0eop_microkappa_example_train)
# instance_d0eop_microkappa_example_train.set_index('datetime', inplace = True)
# instance_d0eop_microkappa_example_train

In [28]:
###### load ml constraint ######
pred_constr_d0eop_microkappa = add_predictor_constr(gp_model = m, 
                                   predictor = model_d0eop_microkappa, 
                                   input_vars = instance_d0eop_microkappa, 
                                   output_vars = microkappa_d1,
                                   name = f'model_predict_d0eop_microkappa'
                                  )
pred_constr_d0eop_microkappa.print_stats()

Model for model_predict_d0eop_microkappa:
72 variables
1 constraints
66 quadratic constraints
Input has shape (1, 10)
Output has shape (1, 1)

Pipeline has 2 steps:

--------------------------------------------------------------------------------
Step            Output Shape    Variables              Constraints              
                                                Linear    Quadratic      General
poly_feat            (1, 66)           72            0           66            0

lin_reg               (1, 1)            0            1            0            0

--------------------------------------------------------------------------------


#### 7.2 d1_brillo

In [29]:
# (reemplazar "d0eop_microkappa" por "d1_brillo")
######################## instance model d1_brillo ########################

# create instance with controlables variables. sorted according the list of features. ES MUY IMPORTANTE QUE ESTÉ ORDENADO LAS VARIABLES DE DECUISIÓN DE ACUERDO A LA LISTA DE FEATURES
instance_controlables_d1_brillo = pd.DataFrame([especifico_dioxido_d1, especifico_acido_d1]).T # <---- change ---------<--------<--------
instance_controlables_d1_brillo.columns = list_features_controlables_d1_brillo # rename columns
instance_controlables_d1_brillo.reset_index(inplace = True)
instance_controlables_d1_brillo.drop(columns = 'index', inplace = True)

# create instance with target of previos model
instance_previos_target_d1_brillo = pd.DataFrame([microkappa_d1]).T # <---- change ---------<--------<--------
instance_previos_target_d1_brillo.columns  = list_target_d0eop_microkappa # rename columns
instance_previos_target_d1_brillo.reset_index(inplace = True)
instance_previos_target_d1_brillo.drop(columns = 'index', inplace = True)

# append features controlables with no controlables
instance_d1_brillo = pd.concat([instance_no_controlables_d1_brillo, instance_controlables_d1_brillo, instance_previos_target_d1_brillo], axis = 1)
instance_d1_brillo = instance_d1_brillo[list_features_d1_brillo] # sort features

# set index - optimization set
instance_d1_brillo.index = index_bleaching

instance_d1_brillo

,240FI108A.PNT,calc_prod_d1,240AIT225A.PNT,240AIT225B.PNT,240TIT223.PNT,240FY218.RO02,240FY210A.RO01
bleaching,169.548609,3348.667487,<gurobi.Var microkappa_d1[bleaching]>,85.050203,77.214156,<gurobi.Var especifico_dioxido_d1[bleaching]>,<gurobi.Var especifico_acido_d1[bleaching]>


In [30]:
###### load ml constraint ######
pred_constr_d1_brillo = add_predictor_constr(gp_model = m, 
                                   predictor = model_d1_brillo, 
                                   input_vars = instance_d1_brillo, 
                                   output_vars = brillo_entrada_p,
                                   name = f'model_predict_d1_brillo'
                                  )

pred_constr_d1_brillo.print_stats()

Model for model_predict_d1_brillo:
4 variables
1 constraints
Input has shape (1, 7)
Output has shape (1, 1)


#### 7.3 model p_blancura

In [31]:
# (reemplazar "d1_brillo" por "p_blancura")
######################## instance model p_blancura ########################

# create instance with controlables variables. sorted according the list of features. ES MUY IMPORTANTE QUE ESTÉ ORDENADO LAS VARIABLES DE DECUISIÓN DE ACUERDO A LA LISTA DE FEATURES
instance_controlables_p_blancura = pd.DataFrame([especifico_soda_p, especifico_peroxido_p, especifico_acido_p]).T # <---- change ---------<--------<--------
instance_controlables_p_blancura.columns = list_features_controlables_p_blancura # rename columns
instance_controlables_p_blancura.reset_index(inplace = True)
instance_controlables_p_blancura.drop(columns = 'index', inplace = True)

# create instance with target of previos model
instance_previos_target_p_blancura = pd.DataFrame([brillo_entrada_p]).T # <---- change ---------<--------<--------
instance_previos_target_p_blancura.columns  = list_target_d1_brillo # rename columns
instance_previos_target_p_blancura.reset_index(inplace = True)
instance_previos_target_p_blancura.drop(columns = 'index', inplace = True)

# append features controlables with no controlables
instance_p_blancura = pd.concat([instance_no_controlables_p_blancura, instance_controlables_p_blancura, instance_previos_target_p_blancura], axis = 1)
instance_p_blancura = instance_p_blancura[list_features_p_blancura] # sort features

# set index - optimization set
instance_p_blancura.index = index_bleaching

instance_p_blancura

,calc_prod_p,240AIT322B.PNT,240AIC324.MEAS,240FY397.RO01,240FY312.RO01,240FY430.RO01
bleaching,3347.782721,<gurobi.Var brillo_entrada_p[bleaching]>,10.382128,<gurobi.Var especifico_soda_p[bleaching]>,<gurobi.Var especifico_peroxido_p[bleaching]>,<gurobi.Var especifico_acido_p[bleaching]>


In [32]:
###### load ml constraint ######
pred_constr_p_blancura = add_predictor_constr(gp_model = m, 
                                   predictor = model_p_blancura, 
                                   input_vars = instance_p_blancura, 
                                   output_vars = blancura_salida_p,
                                   name = f'model_predict_p_blancura'
                                  )

pred_constr_p_blancura.print_stats()

Model for model_predict_p_blancura:
30 variables
1 constraints
28 quadratic constraints
Input has shape (1, 6)
Output has shape (1, 1)

Pipeline has 2 steps:

--------------------------------------------------------------------------------
Step            Output Shape    Variables              Constraints              
                                                Linear    Quadratic      General
poly_feat            (1, 28)           30            0           28            0

lin_reg               (1, 1)            0            1            0            0

--------------------------------------------------------------------------------


#### DOCUMENTATION GUROBI MACHINE LEARNING

Call
[add_predictor_constr](https://gurobi-machinelearning.readthedocs.io/en/stable/auto_generated/gurobi_ml.add_predictor_constr.html)
to insert the constraints linking the features and the demand into the model `m`.

It is important that you keep the columns in the order above, otherwise you will see an error. The columns must be in the same order as the training data.

Obs: to add this constraints the way is little different and it is not neccesary call model.update()

**Documentation - parameters**

- gp_model (gurobipy model) – The gurobipy model where the predictor should be inserted.

- predictor – The predictor to insert.

- input_vars (mvar_array_like) – Decision variables used as input for predictor in gp_model.

- output_vars (mvar_array_like, optional) – Decision variables used as output for predictor in gp_model.

### 8. Define Objetive Function
The goal is to maximize the **net revenue**, which is the product of price and quantity, minus costs over all regions. This model assumes the purchase costs are fixed (since the amount $B$ is fixed) and are therefore not incorporated.

\begin{align} \textrm{maximize} &  \sum_{r}  (p_r * s_r - c_{waste} * u_r -
c^r_{transport} * x_r)& \end{align}

In [33]:
# ######################## define variable of costs of each stage ########################
costs_d0 = especifico_dioxido_d0*prices['dioxido'].values[0]
costs_eop = especifico_soda_eop*prices['soda'].values[0] + especifico_peroxido_eop*prices['peroxido'].values[0] + especifico_oxigeno_eop*prices['oxigeno'].values[0]
costs_d1 = especifico_acido_d1*prices['acido'].values[0] + especifico_dioxido_d1*prices['dioxido'].values[0]
costs_p = especifico_acido_p*prices['acido'].values[0] + especifico_soda_p*prices['soda'].values[0] + especifico_peroxido_p*prices['peroxido'].values[0]

In [34]:
costs_d0

bleaching    1.4 especifico_dioxido_d0[bleaching]
Name: especifico_dioxido_d0, dtype: object

In [35]:
costs_d0.sum()

<gurobi.LinExpr: 1.4 especifico_dioxido_d0[bleaching]>

In [36]:
######################## set objetive minimize costs ########################

# it is necesary define with .sum() to get a guroli linear expression
m.setObjective(costs_d0.sum()+ costs_eop.sum() + costs_d1.sum() + costs_p.sum(),
               gp.GRB.MAXIMIZE)

### 9. Solve optimization problem
The objective is **quadratic** since we take the product of price and the predicted sales, both of which are variables. Maximizing a quadratic
term is said to be **non-convex**, and we specify this by setting the value of the [Gurobi NonConvex
parameter](https://www.gurobi.com/documentation/10.0/refman/nonconvex.html) to be $2$.

#### 9.1 Solve optimization problem

In [37]:
# solve
m.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 10.0 (19043.2))

CPU model: Intel(R) Core(TM) i7-10750H CPU @ 2.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 3 rows, 118 columns and 104 nonzeros
Model fingerprint: 0x2717e8e3
Model has 94 quadratic constraints
Coefficient statistics:
  Matrix range     [9e-09, 3e+01]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 1e+00]
  Objective range  [9e-02, 1e+00]
  Bounds range     [1e+00, 3e+03]
  RHS range        [1e+01, 4e+02]
  QRHS range       [1e+00, 1e+00]
Presolve removed 0 rows and 46 columns

Continuous model is non-convex -- solving as a MIP

Presolve removed 0 rows and 86 columns
Presolve time: 0.00s
Presolved: 75 rows, 32 columns, 198 nonzeros
Presolved model has 20 bilinear constraint(s)
Variable types: 32 continuous, 0 integer (0 binary)
Found heuristic solution: objective 56.8965416

Root relaxation: object

In [38]:
#### know the status of the model - 2 a optimal solution was founded
# docu: https://www.gurobi.com/documentation/current/refman/optimization_status_codes.html#sec:StatusCodes
m.Status

2

#### 9.2 Save optimal values in a dataframe
To get the optimal values of decision variables it is neccesary call "var.gppd.X"

In [39]:
######## create a dataframe with set as index
solution = pd.DataFrame(index = index_bleaching)

######################## save optimal values - features of models (only the features) ########################

# model d0eop_microkappa
solution["especifico_dioxido_d0"] = especifico_dioxido_d0.gppd.X
solution["especifico_oxigeno_eop"] = especifico_oxigeno_eop.gppd.X
solution["especifico_peroxido_eop"] = especifico_peroxido_eop.gppd.X
solution["especifico_soda_eop"] = especifico_soda_eop.gppd.X

# model d1_brillo
solution["especifico_dioxido_d1"] = especifico_dioxido_d1.gppd.X
solution["especifico_acido_d1"] = especifico_acido_d1.gppd.X

# model p_blancura
solution["especifico_soda_p"] = especifico_soda_p.gppd.X
solution["especifico_peroxido_p"] = especifico_peroxido_p.gppd.X
solution["especifico_acido_p"] = especifico_acido_p.gppd.X


######################## save optimal values - targets of models (some targets are features of the model of the next step) ########################
solution["microkappa_d1"] = microkappa_d1.gppd.X  # model d0eop_microkappa
solution["brillo_entrada_p"] = brillo_entrada_p.gppd.X  # model d1_brillo
solution["blancura_salida_p"] = blancura_salida_p.gppd.X  # model p_blancura


######################## round values ########################
solution = solution.round(3)


######################## # get value objetive function ########################
opt_cost = m.ObjVal

In [40]:
# show value objetive function
print("\n The optimal net revenue: $%f K dolars" % opt_cost)


 The optimal net revenue: $58.599276 K dolars


In [41]:
# show value decision variables
solution

,especifico_dioxido_d0,especifico_oxigeno_eop,especifico_peroxido_eop,especifico_soda_eop,especifico_dioxido_d1,especifico_acido_d1,especifico_soda_p,especifico_peroxido_p,especifico_acido_p,microkappa_d1,brillo_entrada_p,blancura_salida_p
bleaching,11.0,2.0,1.558,20.0,10.0,10.0,5.441,10.0,5.579,1.9,90.615,90.0


We can also check the error in the estimate of the Gurobi solution for the regression model.

In [42]:
print(
    "Maximum error in approximating the regression {:.6}".format(
        np.max(pred_constr_d0eop_microkappa.get_error())
    )
)

Maximum error in approximating the regression 7.12916e-09


In [43]:
print(
    "Maximum error in approximating the regression {:.6}".format(
        np.max(pred_constr_d1_brillo.get_error())
    )
)

Maximum error in approximating the regression 0.0


In [44]:
print(
    "Maximum error in approximating the regression {:.6}".format(
        np.max(pred_constr_p_blancura.get_error())
    )
)

Maximum error in approximating the regression 2.73509e-07


In [45]:
# show all constraints
m.getConstrs()

[<gurobi.Constr model_predict_d0eop_microkappa.lin_reg.linreg[0,0]>,
 <gurobi.Constr model_predict_d1_brillo.linreg[0,0]>,
 <gurobi.Constr model_predict_p_blancura.lin_reg.linreg[0,0]>]

### TODO FALTANTES - IDEAS POR HACER PARA MEJORAR EL MODELO
- Agregar restricciones de que el valor óptimo de las variables se puedan mover solamente un delta de un cierto valor que en estado es la condición actual. Para que no se recomienden cambios bruscos que no se puedan conseguir.
- Parametrizar códigos y dejar en una función. Luego si no se encuentra una soluciónn óptima ir relajando las restricciones hasta tener una solución óptima (definir plan para relajar restricciones)
- Agregar temporalidad. Que con el mismo modelo de machine learning se utilizen restricciones de variables de decisión que están definidos en un conjunto temporal

### TODO FALTANTES - TRANSVERSALES EN LOS CÓDIGOS
- Las tablas de parámetros/tablas maestras/whatever que sean por modelo. O quizás tener una global. O quizás tener solo una global. No sé. Pensar. Probablemente tiene que ser por modelo (DONE)
- Agregar el registro de experimentos de Vertex
- Agregar los códigos de validación offline de los modelos de machine learning